<a href="https://colab.research.google.com/github/creative-h/agentQ_Travel_Planner/blob/main/travel_booking_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Travel Package Booking Portal

This notebook demonstrates how to use the travel package booking portal to search for and book travel packages that combine flights and hotels.

## 1. Setup and Configuration

First, let's install the required dependencies:

In [1]:
!git clone https://github.com/creative-h/agentQ_Travel_Planner.git

Cloning into 'agentQ_Travel_Planner'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 6), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 1.47 MiB | 6.69 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
!unzip agentQ_Travel_Planner/travelAgentQ.zip

Archive:  agentQ_Travel_Planner/travelAgentQ.zip
   creating: travelAgentQ/
   creating: travelAgentQ/.git/
 extracting: travelAgentQ/.git/COMMIT_EDITMSG  
 extracting: travelAgentQ/.git/HEAD  
 extracting: travelAgentQ/.git/ORIG_HEAD  
   creating: travelAgentQ/.git/branches/
  inflating: travelAgentQ/.git/config  
  inflating: travelAgentQ/.git/description  
   creating: travelAgentQ/.git/hooks/
  inflating: travelAgentQ/.git/hooks/applypatch-msg.sample  
  inflating: travelAgentQ/.git/hooks/commit-msg.sample  
  inflating: travelAgentQ/.git/hooks/fsmonitor-watchman.sample  
  inflating: travelAgentQ/.git/hooks/post-update.sample  
  inflating: travelAgentQ/.git/hooks/pre-applypatch.sample  
  inflating: travelAgentQ/.git/hooks/pre-commit.sample  
  inflating: travelAgentQ/.git/hooks/pre-merge-commit.sample  
  inflating: travelAgentQ/.git/hooks/pre-push.sample  
  inflating: travelAgentQ/.git/hooks/pre-rebase.sample  
  inflating: travelAgentQ/.git/hooks/pre-receive.sample  
  infla

In [1]:
cd /content/travelAgentQ/travel_booking_portal


/content/travelAgentQ/travel_booking_portal


In [4]:
!pip install "httpx<0.28"
# !pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.16.1 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.16.1 requires websockets<15.1.0,>=13.0.0, but you have websockets 11.0.3 which is incompatible.
langchain-core 0.3.60 requires pydantic>=2.7.4, but you have pydantic 2.5.3 which is incompatible.


Now, let's set up our API credentials. You'll need to obtain API keys for:
- Amadeus (for flight and hotel data): https://developers.amadeus.com/
- Groq (for AI-powered recommendations): https://console.groq.com/

In [2]:
import os
import sys
import logging

# Add parent directory to path to import modules
sys.path.append('..')

# Set up API credentials
os.environ['AMADEUS_API_KEY'] = "bsknoiobYmGP9Ur47t6bGwFDUMihUtnH"  # Replace with your actual API key
os.environ['AMADEUS_API_SECRET'] = "4YN13VQCc0ti4X8I"  # Replace with your actual API secret
os.environ['GROQ_API_KEY'] = "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"  # Replace with your actual API key
os.environ['GROQ_MODEL'] = "llama-3.3-70b-versatile"  # You can change this to a different model if needed

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('notebook')

## 2. Initialize APIs and Agent

Now, let's initialize the APIs and agent for our travel booking portal:

In [3]:
from utils import Config, TravelAgent, DataProcessor
from api import FlightAPI, HotelAPI, PackageAPI

# Load configuration
config = Config()
if not config.validate():
    logger.error("Invalid configuration. Please check your API credentials.")
    raise ValueError("Invalid configuration")

# Initialize APIs
amadeus_credentials = config.get_amadeus_credentials()
flight_api = FlightAPI(amadeus_credentials['api_key'], amadeus_credentials['api_secret'])
hotel_api = HotelAPI(amadeus_credentials['api_key'], amadeus_credentials['api_secret'])
package_api = PackageAPI(flight_api, hotel_api)

# Initialize agent
groq_credentials = config.get_groq_credentials()
travel_agent = TravelAgent(groq_credentials['api_key'], groq_credentials['model'])

# Initialize data processor
data_processor = DataProcessor()

print("APIs and agent initialized successfully!")

APIs and agent initialized successfully!


## 3. Natural Language Query Processing

Let's use the travel agent to process a natural language query and extract travel criteria:

In [5]:
# Example natural language query
query = "I want to travel from New York to Paris for a week in July with my family (2 adults, 1 child)"

# Extract travel criteria
criteria = travel_agent.extract_travel_criteria(query)

# Display extracted criteria
import json
print(json.dumps(criteria, indent=2))

ERROR:agent:Error parsing JSON from response: Extra data: line 14 column 1 (char 257)
ERROR:agent:Raw response: ```json
{
  "origin_location": "New York",
  "destinations": ["Paris"],
  "departure_date": null,
  "return_date": null,
  "adults": 2,
  "children": 1,
  "infants": 0,
  "travel_class": null,
  "budget": null,
  "preferences": null,
  "multi_destination": false
}
```

Note: The departure and return dates are not explicitly mentioned in the query, but it's mentioned that the trip is for a week in July. If you want to include this information, you could add a "departure_month" and "trip_duration" field to the JSON object, like this:

```json
{
  "origin_location": "New York",
  "destinations": ["Paris"],
  "departure_date": null,
  "return_date": null,
  "departure_month": "July",
  "trip_duration": 7,
  "adults": 2,
  "children": 1,
  "infants": 0,
  "travel_class": null,
  "budget": null,
  "preferences": null,
  "multi_destination": false
}
```


{}


## 4. Search for Flights

Now, let's search for flights based on the extracted criteria:

In [6]:
# Set up search parameters
origin = criteria.get('origin_location', 'NYC')  # Default to NYC if not extracted
destination = criteria.get('destinations', 'PAR')  # Default to Paris if not extracted
if isinstance(destination, list):
    destination = destination[0]  # Take the first destination if multiple are extracted

# Set dates (default to next month if not extracted)
from datetime import datetime, timedelta
default_departure = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
default_return = (datetime.now() + timedelta(days=37)).strftime('%Y-%m-%d')
departure_date = criteria.get('departure_date', default_departure)
return_date = criteria.get('return_date', default_return)

# Set passenger counts
adults = criteria.get('adults', 2)
children = criteria.get('children', 1)

# Set travel class
travel_class = criteria.get('travel_class', 'ECONOMY')

# Search for flights
print(f"Searching for flights from {origin} to {destination} on {departure_date} returning {return_date}...")
flight_offers = flight_api.search_flights(
    origin_location_code=origin,
    destination_location_code=destination,
    departure_date=departure_date,
    return_date=return_date,
    adults=adults,
    children=children,
    travel_class=travel_class,
    currency_code="USD",
    max_results=5
)

# Display flight offers
print(f"Found {len(flight_offers)} flight offers")
if flight_offers:
    # Parse flight data into a DataFrame
    flights_df = data_processor.parse_flight_data(flight_offers)
    flights_df.head()

Searching for flights from NYC to PAR on 2025-06-27 returning 2025-07-04...
Found 5 flight offers


## 5. Search for Hotels

Now, let's search for hotels at the destination:

In [7]:
# Search for hotels
print(f"Searching for hotels in {destination} from {departure_date} to {return_date}...")
hotel_offers = hotel_api.search_hotels(
    city_code=destination,
    check_in_date=departure_date,
    check_out_date=return_date,
    adults=adults,
    radius=5,
    radius_unit="KM",
    currency="USD",
    max_results=5
)

# Display hotel offers
print(f"Found {len(hotel_offers)} hotel offers")
if hotel_offers:
    # Parse hotel data into a DataFrame
    hotels_df = data_processor.parse_hotel_data(hotel_offers)
    hotels_df.head()

Searching for hotels in PAR from 2025-06-27 to 2025-07-04...


ERROR:hotel_api:Error searching for hotels: object of type 'NoneType' has no len()


Found 0 hotel offers


## 6. Create Travel Packages

Now, let's combine flights and hotels to create travel packages:

In [8]:
# Create travel packages
print("Creating travel packages...")
packages = package_api.create_single_destination_package(
    origin_location_code=origin,
    destination_location_code=destination,
    departure_date=departure_date,
    return_date=return_date,
    adults=adults,
    children=children,
    travel_class=travel_class,
    currency_code="USD",
    max_flight_results=5,
    max_hotel_results=5
)

# Display packages
print(f"Created {len(packages)} travel packages")
if packages:
    # Parse package data into a DataFrame
    packages_df = data_processor.parse_package_data(packages)
    packages_df.head()

Creating travel packages...


ERROR:hotel_api:Error searching for hotels: object of type 'NoneType' has no len()


Created 0 travel packages


## 7. Get AI-Powered Recommendations

Now, let's use the travel agent to generate personalized recommendations based on the available packages:

In [9]:
# Generate recommendations
if packages:
    print("Generating recommendations...")
    recommendations = travel_agent.generate_travel_recommendations(packages[:3], criteria)
    print("\nRecommendations:")
    print(recommendations)

## 8. View Package Details

Let's view the details of a specific package:

In [10]:
# View package details
if packages:
    package_id = 0  # View the first package
    print(f"Viewing details for package {package_id}...")
    package_details = travel_agent.format_package_details(packages[package_id], detailed=True)
    print("\nPackage Details:")
    print(package_details)

## 9. Ask Travel-Related Questions

Finally, let's ask the travel agent some questions:

In [11]:
# Ask a question
question = "What's the best time to visit Paris?"
print(f"Question: {question}")

# Prepare context with current search results and criteria
context = {
    'current_criteria': criteria,
    'has_search_results': len(packages) > 0,
    'number_of_packages': len(packages) if packages else 0,
    'price_range': {
        'min': min([p['total_price'] for p in packages]) if packages else None,
        'max': max([p['total_price'] for p in packages]) if packages else None,
        'currency': packages[0]['currency'] if packages else None
    }
}

# Get answer from agent
answer = travel_agent.answer_travel_question(question, context)
print("\nAnswer:")
print(answer)

Question: What's the best time to visit Paris?

Answer:
The best time to visit Paris is during the spring (April-May) and autumn (September-October), when the weather is mild and pleasant, with average temperatures ranging from 15°C to 25°C (59°F to 77°F). These periods offer a great balance of comfortable weather and smaller crowds, making it ideal for exploring the city's famous landmarks and attractions.


## 10. Launch Gradio Web Interface

If you want to use the web interface instead of the notebook, you can launch the Gradio app:

In [12]:
from frontend.app import app as gradio_app

# Launch the app
gradio_app.launch()

/usr/local/lib/python3.11/dist-packages/gradio/components/dropdown.py:173: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include:  or set allow_custom_value=True.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/components/dropdown.py:173: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: 2025-05-28 or set allow_custom_value=True.
  warnings.warn(


IMPORTANT: You are using gradio version 4.19.2, however version 4.44.1 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1ef62858c470233fdd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
